In [5]:


import pandas
import os
import zipfile
import requests
import tqdm
import numpy as np
from sklearn import dummy, metrics, cross_validation, ensemble

import keras.models as kmodels
import keras.layers as klayers
import keras.backend as K
import keras


if not os.path.exists('./ml-1m'):
    url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
    response = requests.get(url, stream=True)
    total_length = response.headers.get('content-length')
    bar = tqdm.tqdm_notebook(total=int(total_length))
    with open('./ml-1m.zip', 'wb') as f:
        for data in response.iter_content(chunk_size=4096):
            f.write(data)
            bar.update(4096)
    zip_ref = zipfile.ZipFile('./ml-1m.zip', 'r')
    zip_ref.extractall('.')
    zip_ref.close()

In [7]:
# Read in the dataset, and do a little preprocessing,
# mostly to set the column datatypes.
users = pandas.read_csv('./ml-1m/users.dat', sep='::', 
                        engine='python', 
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('./ml-1m/ratings.dat', engine='python', 
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies = pandas.read_csv('./ml-1m/movies.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre']).set_index('movieid')
movies['genre'] = movies.genre.str.split('|')
users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')

In [8]:
# Count the movies and users
n_movies = movies.shape[0]
n_users = users.shape[0]

# Also, make vectors of all the movie ids and user ids. These are
# pandas categorical data, so they range from 1 to n_movies and 1 to n_users, respectively.
movieid = ratings.movieid.cat.codes.values
userid = ratings.userid.cat.codes.values


In [37]:
from keras.models import Sequential 
from keras import preprocessing
from keras.layers import Embedding
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import Embedding, Flatten, Dense
from keras.layers import SimpleRNN
from keras.preprocessing import sequence
from keras.layers import Dense, Activation
from keras.layers import Input, LSTM, Dense
from keras.layers.normalization import BatchNormalization
#Get labels
y = np.zeros((ratings.shape[0], 5))
y[np.arange(ratings.shape[0]), ratings.rating - 1] = 1

dout = 1.0

movie_input = keras.layers.Input(shape=[1])
movie_vec = keras.layers.Flatten()(keras.layers.Embedding(n_movies + 1, 32)(movie_input))
movie_vec = keras.layers.Dropout(dout)(movie_vec)

# Same thing for the users
user_input = keras.layers.Input(shape=[1])
user_vec = keras.layers.Flatten()(keras.layers.Embedding(n_users + 1, 32)(user_input))
user_vec = keras.layers.Dropout(dout)(user_vec)

# BUILDING THE LSTM
model = LSTM(64)
#Create input layer
input_vecs = keras.layers.merge([movie_vec, user_vec], mode='concat')
layer = Dense(128,activation='relu')(input_vecs)
layer = Dropout(dout)(BatchNormalization()(layer))
layer = Dense(128,activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dense(128,activation='relu')(layer)
layer = BatchNormalization()(layer)
final_layer = Dense(5,activation='softmax')(layer)

# Finally, we pull out the result!

model = kmodels.Model([movie_input, user_input], final_layer)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
a_movieid, b_movieid, a_userid, b_userid, a_y, b_y = cross_validation.train_test_split(movieid, userid, y)


history = model.fit([a_movieid, a_userid], a_y, 
                         epochs=1)
                        #validation_data=()



/Users/pascal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/pascal/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Epoch 1/1


    32/750156 [..............................] - ETA: 10:50:35 - loss: 0.5695 - acc: 0.7563

   224/750156 [..............................] - ETA: 1:36:03 - loss: 0.6657 - acc: 0.7330 

   352/750156 [..............................] - ETA: 1:03:03 - loss: 0.6500 - acc: 0.7386

...

749600/750156 [============================>.] - ETA: 0s - loss: 0.4213 - acc: 0.8036

749792/750156 [============================>.] - ETA: 0s - loss: 0.4213 - acc: 0.8036

749984/750156 [============================>.] - ETA: 0s - loss: 0.4213 - acc: 0.8036

750144/750156 [============================>.] - ETA: 0s - loss: 0.4213 - acc: 0.8036

750156/750156 [==============================] - 242s 322us/step - loss: 0.4213 - acc: 0.8036


In [38]:
loss1,acc1= model.evaluate([b_movieid, b_userid], b_y)
print(loss1)
print(acc1)

import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

    32/250053 [..............................] - ETA: 2:20:15

   704/250053 [..............................] - ETA: 6:39   

  1312/250053 [..............................] - ETA: 3:43

  1760/250053 [..............................] - ETA: 2:53



...

248768/250053 [============================>.] - ETA: 0s

249536/250053 [============================>.] - ETA: 0s

250053/250053 [==============================] - 20s 79us/step


0.405638805177
0.808137518216


KeyError: 'val_acc'